In [3]:
import logging
from kiteconnect import KiteConnect

#logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="b2w0sfnr1zr92nxm")
kite.login_url()


'https://kite.trade/connect/login?api_key=b2w0sfnr1zr92nxm&v=3'

In [4]:
import platform
from selenium import webdriver
if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
else:
    driver = webdriver.Chrome("./automation/chromedriver")
driver.get(kite.login_url())

In [5]:
driver.current_url

'http://localhost:8888/view/redirect.html?status=success&request_token=aGyPiJQNxdnOJG4oTHZSa1CQE8M0jIC6&action=login'

In [6]:
import re
request_token = re.findall("request_token=(.*)&action",driver.current_url)

In [7]:
request_token[0]

'aGyPiJQNxdnOJG4oTHZSa1CQE8M0jIC6'

In [8]:
# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.

data = kite.generate_session(request_token[0], api_secret="jtga2mp2e5fn29h8w0pe2kb722g3dh1q")
kite.set_access_token(data["access_token"])


In [9]:
import pandas as pd

df = pd.DataFrame(data=kite.instruments())

In [10]:
nse_instruments = df[(df['instrument_type']=='EQ') & (df['exchange']=='NSE')].set_index('tradingsymbol')
nse_instruments.loc['HDFC','instrument_token']

340481

In [34]:
import datetime as dt

symbol = 'HDFC'
myDate="31_Mar_2019"

raw_data = pd.DataFrame(data=kite.historical_data(nse_instruments.loc[symbol,'instrument_token'],dt.datetime(2019,1,1),dt.datetime(2019,4,4), 'minute', continuous=False))
#raw_data.columns = ['Date','Close','High','Low','Open','Volume']
raw_data = raw_data.set_index('date').tz_localize(None)
                

In [35]:
filename ="./kite_data/"+symbol+"_"+myDate+".csv"

raw_data.to_csv(path_or_buf=filename, sep=',', header=True, index=True)

In [37]:
source = "CSV"
data_Source = "KITE"

# filter
days = 5

# frequency
freq = "1min"
price = raw_data[raw_data.index> dt.datetime.now() - dt.timedelta(days = days)]
#price = raw_data
price['adjClose'] = raw_data['close']
#%run "freedom_indicators_calculation.ipynb"

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [38]:
price = price.resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'adjClose':{'adjClose':'last'},'volume':{'volume':'sum'}})

price.dropna(inplace=True)
price.columns = price.columns.droplevel() 

chandlierMultiplier = 1
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning:

using a dict with renaming is deprecated and will be removed in a future version



In [39]:
import numpy as np
from talib import RSI, BBANDS, MACD, AROON, STOCH, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX
from talib import CDLABANDONEDBABY, CDL3BLACKCROWS,CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR, CDLGRAVESTONEDOJI, CDLHAMMER, CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLMARUBOZU,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLSPINNINGTOP,CDL3BLACKCROWS, CDL3LINESTRIKE

price['macd'], price['macdsignal'], price['macdhist'] = MACD(price.adjClose, fastperiod=12, slowperiod=26, signalperiod=9)

price['RSI'] = RSI(price.adjClose, timeperiod=14)

price['slowk'], price['slowd'] = STOCH(price.high, price.low, price.close)

price['aroondown'], price['aroonup'] = AROON(price.high, price.low)

price['atr'] = ATR(price.high, price.low, price.close, timeperiod=22)

price['bbup'], price['bbmid'], price['bblow'] = BBANDS(price.close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

price['obv'] = OBV(np.linspace(1,100,price.index.shape[0]), price.volume)

price['adosc'] = ADOSC(price.high, price.low, price.close, price.volume)

price['ADX'] = ADX(price.high, price.low, price.close)
price['DI_P'] = PLUS_DI(price.high, price.low, price.close)
price['DI_M'] = MINUS_DI(price.high, price.low, price.close)

# Heikin-Ashi
price['HAopen'] = (price.open.shift(1) + price.close.shift(1))/2
price['HAhigh'] = price[['high','open','close']].max(axis = 1, skipna = True)
price['HAlow'] = price[['low','open','close']].min(axis = 1, skipna = True)
price['HAclose'] = (price.open+price.high+price.low+price.close)/4

price['chandlierLong'] = price.close-chandlierMultiplier*price.atr
price['chandlierShort'] = price.close+chandlierMultiplier*price.atr

In [40]:
# Pivot points

PP = (price.high+price.low+price.close)/3
R1 = 2 * PP - price.low
S1 = 2 * PP - price.high

R2 = PP + (price.high - price.low)
S2 = PP - (price.high - price.low)

R3 = price.high + 2 * (PP - price.low)
S3 = price.low - 2 * (price.high - PP)

In [41]:
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC
########## Algorithm #############
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0
    
class Trend(Enum):
    SIDE = 0
    UP = 1
    SUP = 2
    DOWN = 3
    SDOWN = 4

# Math
price['priceSD'] = STDDEV(price.close)
price['priceVAR'] = VAR(price.close)

price['delBBUP'] = (price.bbup - price.close)/(price.bbup-price.bbmid)
price['delBBLOW'] = (price.close - price.bblow)/(price.bbmid-price.bblow)

price['d1Slope'] = np.arctan(price.close - price.close.shift(1)) * 180 / np.pi
price['d3Slope'] = np.arctan(price.close - price.close.shift(3)) * 180 / np.pi
price['d5Slope'] = np.arctan(price.close - price.close.shift(5)) * 180 / np.pi
price['d10Slope'] = np.arctan(price.bbmid - price.bbmid.shift(3)) * 180 / np.pi
price['d15Slope'] = np.arctan(price.close - price.close.shift(15)) * 180 / np.pi

price['macd1dSlope'] = np.arctan(price.macd - price.macd.shift(1)) * 180 / np.pi

price['macdTSF'] = TSF(price.macd)
price['priceTSF'] = TSF(price.close)


price['ROC'] = ROC(price.close)
price['macdROC'] = ROC(price.macd)

TrendThreshold = 85
trend = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
trend[price.d10Slope >= TrendThreshold] = Trend.UP
trend[price.d10Slope <= -1 * TrendThreshold] = Trend.DOWN
#price.trend1D[price1dGradient >= 65] = Trend.SUP
#price.trend1D[price1dGradient <= -65] = Trend.SDOWN

price['trend1D'] = trend

price['sdHighLow'] = STDDEV((price.high - price.close))
price['varHighLow'] = STDDEV((price.high - price.close))
price['delPriceHighLow'] = (price.high - price.close)/price.sdHighLow

price['PriceChange%'] = (price.close - price.shift(1).close)*100/price.close

price.dropna(inplace=True)
labels = pd.DataFrame(data = {"holdings":np.full(price.index.shape[0], S.HOLD)},index=price.index)
    

In [42]:
buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]

In [44]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [58]:
price.close[-1]


2008.75

In [60]:

lrange = price.close[-1]-30
rrange = price.close[-1]+30

#################################
#   Plotting Data              #
###############################

rangeMin = price.index[-150]
rangeMax = price.index[-1]

# Price vs date curve with green and red dots to indicate buy and sell points

traceBuy = go.Scatter(x=buy.index, y=buy.adjClose, marker = dict(color='green', size=8), mode='markers', name="Buy", showlegend=False)
traceSell = go.Scatter(x=sell.index, y=sell.adjClose, marker = dict(color='red', size=8), mode='markers', name="Sell", showlegend=False)


# Candlestick
traceOHLC = go.Candlestick(x=price.index, open=price['open'], high=price['high'], low=price['low'], close=price['close'], name="Candlestick", showlegend=False)

# Chandlier Exit - Long
traceSL = go.Scatter(x=price.index, y=price.chandlierLong, marker = dict(color='lightgrey', size=2), mode='lines', name="Chandlier", showlegend=False)
# Chandlier Exit - Short
traceSL2 = go.Scatter(x=price.index, y=price.chandlierShort, marker = dict(color='lightgrey', size=2), mode='lines', name="Chandlier", showlegend=False)


# Closing Price line chart
tracePrice = go.Scatter(x=price.index, y=price.adjClose, marker = dict(color='grey', size=2), mode='lines', name="Close Price", yaxis='y1', showlegend=False)
tracePricetsf = go.Scatter(x=price.index, y=price.macdTSF, marker = dict(color='blue', size=2), mode='lines', name="Close Price", yaxis='y1', showlegend=False)


# BBP
traceBBU = go.Scatter(x=price.index, y=price['bbup'], name='BB_up',  line=dict(color='lightgrey'),showlegend=False)
traceBBL = go.Scatter(x=price.index, y=price['bblow'], name='BB_low',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
traceBBM = go.Scatter(x=price.index, y=price.bbmid, name='BB_mid',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)

# RSI plots
traceRSI = go.Scatter(x=price.index, y=price['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False)

# ATR
traceATR = go.Scatter(x=price.index, y=price['atr'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='ATR',showlegend=False)


# MACD plots
traceMACD = go.Scatter(x=price.index, y=price['macd'], name='MACD', line=dict(color='black'),showlegend=False)
traceMACDSignal = go.Scatter(x=price.index, y=price['macdsignal'], name='MACD signal', line=dict(color='red'),showlegend=False)
traceMACDHist = go.Bar(x=price.index, y=price['macdhist'], name='MACD Hist', marker=dict(color="grey"),showlegend=False)

# Volume Chart
traceV = go.Bar(x=price.index, y=price.volume, name='Volume', marker=dict(color='lightgrey'),yaxis='y2',showlegend=False)
traceOBV = go.Scatter(x=price.index, y=price['obv'], name='OBV', line=dict(color='black'),showlegend=False)



# Stochastics

traceSK  = go.Scatter(x=price.index, y=price['slowk'], name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
traceSD  = go.Scatter(x=price.index, y=price['slowd'], name='%D', line=dict(color='red'),showlegend=False)

# AROON
traceARD  = go.Scatter(x=price.index, y=price['aroondown'], name='AROON Down', line=dict(color='red'), yaxis='y4',showlegend=False)
traceARU  = go.Scatter(x=price.index, y=price['aroonup'], name='AROON UP', line=dict(color='green'),showlegend=False)

# Haikin Ashi
traceHA = go.Candlestick(x=price.index, open=price['HAopen'], high=price['HAhigh'], low=price['HAlow'], close=price['HAclose'], name="Heikin Ashi", showlegend=False)

# ADX
traceADX  = go.Scatter(x=price.index, y=price['ADX'], name='ADX', line=dict(color='blue') ,showlegend=False)
traceDIP  = go.Scatter(x=price.index, y=price['DI_P'], name='DI+', line=dict(color='green'),showlegend=False)
traceDIM  = go.Scatter(x=price.index, y=price['DI_M'], name='DI-', line=dict(color='red'),showlegend=False)


fig = tools.make_subplots(rows=9, cols=1, row_width=[4,1,1,2,2,3,1,4,4], shared_xaxes=True)

# OHLC
fig.append_trace(traceOHLC, 1, 1)

# Chandlier
fig.append_trace(traceSL, 1, 1)
fig.append_trace(traceSL2, 1, 1)

# Price
fig.append_trace(tracePrice, 2, 1)
#fig.append_trace(tracePricetsf, 4, 1)
fig.append_trace(traceBuy, 2, 1)
fig.append_trace(traceSell, 2, 1)

#BB
fig.append_trace(traceBBU, 2, 1)
fig.append_trace(traceBBM, 2, 1)
fig.append_trace(traceBBL, 2, 1)

#Volume
fig.append_trace(traceV,3,1)
#fig.append_trace(traceOBV,2,1)

# MACD
fig.append_trace(traceMACD, 4, 1)
fig.append_trace(traceMACDSignal, 4, 1)
fig.append_trace(traceMACDHist, 4, 1)

# Stochastics
fig.append_trace(traceSK, 6, 1)
fig.append_trace(traceSD, 6, 1)

# RSI
fig.append_trace(traceRSI, 7, 1)

# ATR
#fig.append_trace(traceATR, 6, 1)

# AARON
#fig.append_trace(traceARD, 8, 1)
#fig.append_trace(traceARU, 8, 1)

# AARON
fig.append_trace(traceADX, 8, 1)
fig.append_trace(traceDIP, 8, 1)
fig.append_trace(traceDIM, 8, 1)

# Heikin-Ashi
fig.append_trace(traceHA, 9, 1)


# Update layout
fig['layout'].update(height=1050, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)

fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), range =[rangeMin,rangeMax], side="top")
fig['layout']['legend'] = dict(orientation="h")

fig['layout']['yaxis1'] = dict( side='left', domain=[0.75,1], showgrid=True , range=[lrange,rrange]
                              #  tickvals=[PP[-1],S1[-1],S2[-1],S3[-1],R1[-1],R2[-1],R3[-1]],
                              #  ticktext=[PP[-1],S1[-1],S2[-1],S3[-1],R1[-1],R2[-1],R3[-1]], tickcolor='#000'
                              )

fig['layout']['yaxis2'] = dict( side='right', domain=[0.55,0.90], showgrid=True, range=[lrange,rrange])
fig['layout']['yaxis3'] = dict( side='left',  domain=[0.55,0.65], showgrid=False)
fig['layout']['yaxis4'] = dict( side='right', domain=[0.40,0.55], title="MACD",visible=False,)
fig['layout']['yaxis5'] = dict( side='right', scaleanchor = "y4", scaleratio = 1.5, visible=False, showgrid=False)
fig['layout']['yaxis6'] = dict( side='right', domain=[0.32,0.42], title="Stochastics", tickvals=[80,70,30,20], ticktext=[80,70,30,20])
fig['layout']['yaxis7'] = dict( side='right', domain=[0.27,0.32], title="RSI", tickvals=[80,70,30,20], ticktext=[80,70,30,20])
fig['layout']['yaxis8'] = dict( side='right', domain=[0.2,0.25], title="AROON")
fig['layout']['yaxis9'] = dict( side='right', domain=[0,0.2], title="Heikin Ashi", range=[lrange,rrange])


fig['layout']['xaxis']['rangeselector'] = dict(
            buttons=list([dict(count=1, label='1m', step='minute', stepmode='backward'),
                          dict(count=3, label='3m', step='minute', stepmode='backward'),
                          dict(count=5, label='5m', step='minute', stepmode='backward'),
                          dict(count=1, label='1h', step='hour', stepmode='backward'),
                          dict(count=3, label='3h', step='hour', stepmode='backward'),
                          dict(count=1, label='1d', step='day', stepmode='todate'),
                          dict(step='all')]))

                     
iplot(fig, filename="Test")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]
[ (7,1) x1,y7 ]
[ (8,1) x1,y8 ]
[ (9,1) x1,y9 ]

